# Integrating Metadata into Annotated Book Review Articles

In this notebook, we will load the exported Label Studio annotated data for the Trouw and Het Parool newspapers and enrich it by adding metadata. This metadata includes important information about the book review articles, such as the year of publication.

The main steps in this process are:
1. **Data Loading:** Import the annotated data from Label Studio and the metadata from the newspapers
2. **Metadata Integration:** Merge metadata to annotations, specifically the year of publication, to each annotated book review article.
3. **Data Preparation:** Prepare the enriched dataset for further analysis or modeling.

By the end of this notebook, we will have an annotated dataset that includes both the annotations and relevant metadata for the book review articles from Trouw and Het Parool.

Let's get started!

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json

In [4]:
def get_csv(file_path):
    return pd.read_csv(file_path, delimiter=';').drop(columns=['Unnamed: 0'])

def load_dataframes(base_path, file_prefix, start_year, end_year):
    dataframes = []
    for year in tqdm(range(start_year, end_year + 1)):
        file_path = f"{base_path}/{file_prefix}_{year}.csv"
        df = get_csv(file_path)
        dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)

base_path = "D:/UU/Kranten"

# Load dataframes for each newspaper
parool_df = load_dataframes(base_path, "Parool", 1945, 1995)
trouw_df = load_dataframes(base_path, "Trouw", 1946, 1995)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:14<00:00,  1.49s/it]


In [5]:
# Merge all dataframes into one big dataframe
all_news_df = pd.concat([parool_df, trouw_df], ignore_index=True)

In [6]:
# Clean up RAM
del parool_df, trouw_df

In [7]:
all_news_df.newspaper.unique()

array(['Parool', 'Trouw'], dtype=object)

In [8]:
all_news_df.genre.unique()

array(['artikel', 'advertentie', 'familiebericht',
       'illustratie met onderschrift'], dtype=object)

In [9]:
all_news_df = all_news_df[all_news_df['genre'] == 'artikel']

In [36]:
# Load the JSON data
with open('data/trouw_label_studio_export.json', 'r', encoding='utf-8') as file:
    trouw_data = json.load(file)

# Load the JSON data
with open('data/parool_label_studio_export.json', 'r', encoding='utf-8') as file:
    parool_data = json.load(file)

In [37]:
trouw_data[0]

{'text': "Macabere trekken bij Cortázar In een van zijn dagboeken schrijft de Poolse Argentijn Witold Gombrowicz hoe moeilijk het is voor schrijvers die in Zuid-Amerika wonen om zowel in eigen land als in Europa naam te maken. Pas wanneer de Parijse literaire wereld een schrijver geaccepteerd heeft, kan deze op sukses in eigen land.— en tegelijkertijd daarbuiten — rekenen. Misschien is dat een reden waarom de Zuidamerikaanse literatuur hier te lande vrij onbekend is. Daar komt de laatste jaren wel verandering in. Als een monument in de huidige wereldliteratuur staat de Argentijn Jorge Luis Borges. Toen deze een paar maanden geleden in Londen lezingen hield, organiseerde men vanuit Nederland zelfs een charterbedevaart. In het Rotterdamse poëzie-festival kon het Nederlandse publiek kennis maken met de in Parijs wonende Chileense dichter Pablo Neruda, die regelmatig op één lijn gesteld wordt met Ezra Pound en T.S. Eliot. Toch is waarschijnlijk het werk van Julio Cortazar in Nederland door

In [41]:
f"number of trouw book reviews: {len(trouw_data)}, and in the parool: {len(parool_data)}"

'number of trouw book reviews: 115, and in the parool: 193'

In [42]:
data = trouw_data + parool_data

In [45]:
# Initialize lists to store the dataframe data
texts = []
start_indices = []
end_indices = []
book_titles = []

# Loop through each entry in the JSON data
for entry in data:
    text = entry['text']
    for label in entry['label']:
        texts.append(text)
        start_indices.append(label['start'])
        end_indices.append(label['end'])
        book_titles.append(label['text'])

# Create the dataframe
df = pd.DataFrame({
    'text': texts,
    'start_index': start_indices,
    'end_index': end_indices,
    'book_title': book_titles
})

In [48]:
final_df = pd.merge(df, all_news_df, left_on='text', right_on='Content', how='left').drop(columns=['Content'])
final_df

,text,start_index,end_index,book_title,RecId,genre,date,year,article_tit,newspaper,month
0,Macabere trekken bij Cortázar In een van zijn ...,0,29,Macabere trekken bij Cortázar,ddd:010818290:mpeg21:a0275,artikel,1971-11-13,1971,NaN,Trouw,11.0
1,Macabere trekken bij Cortázar In een van zijn ...,1651,1664,'Het Gerucht',ddd:010818290:mpeg21:a0275,artikel,1971-11-13,1971,NaN,Trouw,11.0
2,Macabere trekken bij Cortázar In een van zijn ...,5346,5351,Circe,ddd:010818290:mpeg21:a0275,artikel,1971-11-13,1971,NaN,Trouw,11.0
3,'Ik ben altijd gelijk’ door Ad Zuiderent J. Be...,73,83,Het verlof,ddd:010828144:mpeg21:a0319,artikel,1971-09-18,1971,NaN,Trouw,9.0
4,'Ik ben altijd gelijk’ door Ad Zuiderent J. Be...,5601,5611,Het verlof,ddd:010828144:mpeg21:a0319,artikel,1971-09-18,1971,NaN,Trouw,9.0
...,...,...,...,...,...,...,...,...,...,...,...
531,FINANCIEN & ECONOMIE Hoge inkomens hebben mees...,7615,7643,Profijt van de overheid 111.,ddd:010842199:mpeg21:a0297,artikel,1994-12-24,1994,NaN,Parool,12.0
532,BART TROMP De contramine voorbij ER ZIJN TWEE ...,5446,5469,Ik kan alles uitleggen.,ddd:010842201:mpeg21:a0117,artikel,1994-12-28,1994,NaN,Parool,12.0
533,BART TROMP Onaanvaardbaar geworden? zou opbouw...,3886,3953,Jakhalzen van het Derde Rijk. Ondergang van de...,ddd:010841825:mpeg21:a0124,artikel,1995-10-25,1995,NaN,Parool,10.0
534,Dames met sappige tomatentassen Handtasjes heb...,9640,9667,"Rond de tafel der Romeinen,",ddd:010842165:mpeg21:a0173,artikel,1995-01-04,1995,NaN,Parool,1.0


In [50]:
final_df.to_csv('data/trouw_and_parool_annotated_book_titles.csv', index=False)